In [13]:
import sys
import os


# path = "C:/Users/meide/Documents/GitHub/Master/data"
path = "data/"
os.chdir(path)


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'data/'

In [14]:
import json
import jsonschema
import escher

In [63]:
with open('Histidine_lol.json.cyjs') as elements_json:
    abc_cytoscape = json.load(elements_json)
    
with open("escher_jsonschema.1-0-0.json") as escher_schema:
    escher_schema = json.load(escher_schema)

with open('ABC_escher_map.json') as escher_json:
    abc_escher = json.load(escher_json)

jsonschema.validate(abc_escher, escher_schema)

# Cytoscape Model

In [64]:
class cytoscape_model:
    def __init__(self, cyjson):
        if type(cyjson) is str:
            with open(cyjson) as cy:
                self.cy = json.load(cy)
        else:
            self.cy = cyjson
    def get_metabolites( self ):
        return [cytoscape_node(self, node) for node in self.cy['elements']['nodes'] 
                if 'classes' in node and 'cpd' in node['classes']]
    def get_node_ids( self ):
        return [node['data']['id'] for node in self.cy['elements']['nodes']]

    def get_edge_ids( self ):
        return [edge['data']['id'] for edge in self.cy['elements']['edges']]
    def get_node( self, node_id ):
        try:
            for node in self.cy['elements']['nodes']:
                if 'data' in node and 'id' in node['data'] and node['data']['id'] == node_id:
                    return cytoscape_node(self, node)
            return cytoscape_node( self, dict(data=dict(id=node_id )))
        except:
            raise KeyError
    def get_node_by_frame_id( self, frame_id ):
        try:
            for node in self.cy['elements']['nodes']:
                if 'data' in node and 'SUID' in node['data'] and node['data']['SUID'] == frame_id:
                    return cytoscape_node(self, node)
        except:
            raise KeyError
    def get_map_id( self ):
        return self.cy['elements']['nodes'][0]['data']
    def get_reactions( self ):
        """Reactions are collections of edges with the same SUID"""
        rxns = {}
        for edge in self.cy['elements']['edges']:
            if edge['data']['SUID'] in rxns:
                rxns[edge['data']['SUID']].append(cytoscape_edge( self, edge) )
            else:
                rxns[edge['data']['SUID']] = [cytoscape_edge( self, edge ) ]
        return [cytoscape_reaction(self, rxns[frame_id] ) for frame_id in rxns] 
    def get_knots( self ):
        """Knots are nodes that correspond to midmarkers and multimarkers"""
        return [cytoscape_node(self, node) for node in self.cy['elements']['nodes'] 
                if 'classes' in node and 'knot' in node['classes']]

In [65]:
cmodel = cytoscape_model(abc_cytoscape)

In [78]:
class cytoscape_node:
    def __init__(self, cytoscape, node):
        """A cytoscape node knows the model to which it belongs
        """
        self.node = node
        self.cy = cytoscape

    def is_metabolite_node( self ):
        return 'classes' in self.node and 'cpd' in  self.node['classes']
    
    def is_knot_node( self ):
        return 'classes' in self.node and 'knot' in self.node['classes']
    
    def get_node_id( self ):
        return self.node['data']['id']
    
    def get_frame_id( self ):
        return self.node['data']['SUID']     
    def get_label( self ):
        return self.node['data']['label']
    
    def get_x(self ):
        return self.node['position']['x']
    
    def get_y(self):
        return self.node['position']['y']

In [25]:
cmodel = cytoscape_node(abc_cytoscape)

TypeError: __init__() missing 1 required positional argument: 'node'

In [81]:
import copy
class cytoscape_reaction:

    def __init__( self, cytoscape, edges ):
        """A cytoscape reaction knows the model to which it belongs, and the edges that belong to it"""
        self.edges = edges
        self.cy = cytoscape
    def get_frame_id( self ):
        return self.edges[0].get_frame_id()
    def get_left( self ):
        return self.edges[0].get_left()
    def get_right( self ):
        return self.edges[0].get_right()
    def get_edges( self ):
        return copy.deepcopy(self.edges)
    def get_gene_reaction_rule( self ):
        return ' or '.join( sorted( [gene['bigg_id'] for gene in self.get_genes() ] ) )
    def get_name( self ):
        return self.edges[0].get_name()
    def get_rxn_reversibility( self ):
        for edge in self.edges:
            if edge.has_source_arrow():
                return True
        return False
    def get_nodes_of_reaction( self ):
        return [edge.get_target() for edge in self.edges] + [edge.get_source() for edge in self.edges]
    def get_mid_x( self ):
        """Convenience function to compute the x midpoint of a reaction"""
        min_x = min([node.get_x() for node in self.get_nodes_of_reaction()])
        max_x = max([node.get_x() for node in self.get_nodes_of_reaction()])
        return (max_x + min_x)/2.0
    def get_mid_y( self ):
        """Convenience function to compute the y midpoint of a reaction"""

        min_y = min([node.get_y() for node in self.get_nodes_of_reaction()])
        max_y = max([node.get_y() for node in self.get_nodes_of_reaction()])
        return (max_y + min_y)/2.0

In [82]:
class cytoscape_edge:
    def __init__( self, cytoscape, edge ):
        """A cytoscape edge knows the model to which it belongs"""
        self.edge = edge
        self.cy = cytoscape
    def get_edge_id( self ):
        return self.edge['data']['id']
    def get_frame_id( self ):
        return self.edge['data']['SUID']
    def has_source_arrow( self ):
        return self.edge['data']['srcArrow'] == 'triangle'
    def has_target_arrow( self ):
        return self.edge['data']['tgtArrow'] == 'triangle'
    def get_source( self ):
        return self.cy.get_node( self.edge['data']['source'])
    def get_target( self ):
        return self.cy.get_node( self.edge['data']['target'] )
    def get_left( self ):
        return [self.cy.get_node_by_frame_id( cpd['id'] ) for cpd in self.edge['data']['left']]
    def get_right( self ):
        return [self.cy.get_node_by_frame_id( cpd['id'] ) for cpd in self.edge['data']['right']]
    def get_mid_x( self ):
        min_x = min(self.get_source().get_x(), self.get_target().get_x())
        max_x = max(self.get_source().get_x(), self.get_target().get_x())
        return (min_x + max_x)/2.0
    def get_mid_y( self ):
        min_y = min(self.get_source().get_y(), self.get_target().get_y())
        max_y = max(self.get_source().get_y(), self.get_target().get_y())
        return (min_y + max_y)/2.0

# Escher Model

In [83]:
class escher_model:
    def __init__( self, json_file=None):
        if json_file:
            with open(json_file) as escher_json:
                self.escher = json.load(escher_json)
        else:
            self.escher = [{},{}]
    def set_boilerplate( self, map_id,
                        homepage='https://escher.github.io',
                        map_description='',
                        map_name='new_map',
                        schema="https://escher.github.io/escher/jsonschema/1-0-0#",
                        canvas_width=3228,
                       canvas_height=5406,
                       canvas_x=-1076,
                       canvas_y=-1802):
        self.escher[0] = dict(map_id=map_id,
                             homepage=homepage,
                             map_description=map_description,
                             map_name=map_name,
                             schema=schema)
        self.escher[1] = dict(canvas=dict(width=canvas_width,
                                          height=canvas_height,
                                          x=canvas_x,
                                          y=canvas_y),
                             nodes=dict(),
                             reactions=dict(),
                             text_labels= dict())
    def add_node(self, escher_node):
        self.escher[1]['nodes'].update(escher_node.to_dict())
    def add_reaction( self, escher_rxn ):
        self.escher[1]['reactions'].update(escher_rxn.to_dict())
    def to_dict( self ):
        return copy.deepcopy(self.escher)
    

In [70]:
class escher_model:
    def __init__( self, json_file=None):
        if json_file:
            with open(json_file) as escher_json:
                self.escher = json.load(escher_json)
        else:
            self.escher = [{},{}]
    def set_boilerplate( self, map_id,
                        homepage='https://escher.github.io',
                        map_description='',
                        map_name='new_map',
                        schema="https://escher.github.io/escher/jsonschema/1-0-0#",
                        canvas_width=3228,
                       canvas_height=5406,
                       canvas_x=-1076,
                       canvas_y=-1802):
        self.escher[0] = dict(map_id=map_id,
                             homepage=homepage,
                             map_description=map_description,
                             map_name=map_name,
                             schema=schema)
        self.escher[1] = dict(canvas=dict(width=canvas_width,
                                          height=canvas_height,
                                          x=canvas_x,
                                          y=canvas_y),
                             nodes=dict(),
                             reactions=dict(),
                             text_labels= dict())
    def add_node(self, escher_node):
        self.escher[1]['nodes'].update(escher_node.to_dict())
    def add_reaction( self, escher_rxn ):
        self.escher[1]['reactions'].update(escher_rxn.to_dict())
    def to_dict( self ):
        return copy.deepcopy(self.escher)
    

In [71]:
class escher_node:
    def __init__( self, node_id, **args):
        self.node = {node_id: args}
    def to_dict( self ):
        return copy.deepcopy(self.node)

In [72]:
class escher_reaction:
    def __init__( self, rxn_id, **args ):
        self.rxn = {rxn_id: args}
    def to_dict( self ):
        return copy.deepcopy(self.rxn)
  

In [73]:
class escher_segment:
    def __init__(self, segment_id, **args ):
        self.segment = {segment_id: args }
    def to_dict( self ):
        return copy.deepcopy(self.segment)

# Converter


In [80]:
class cytoscape2escher:
    def __init__(self, cytoscape ):
        if type(cytoscape) is dict:
            self.cy = cytoscape_model(cytoscape)
        else:
            self.cy = cytoscape
            
        self.c2e = {}
        self.e2c = {}
        self.eId = 1
        for nodeId in self.cy.get_node_ids():
            self.convert_id( nodeId )
        for rxnId in self.cy.get_reactions():
            self.convert_id( rxnId )
        for edgeId in self.cy.get_edge_ids():
            self.convert_id( edgeId )
        self.escher = escher_model()
        self.escher.set_boilerplate(self.cy.get_map_id())
    def convert_id( self, cId ):
        """memoizes cytoscape id to escher_id mappings"""
        if cId in self.c2e:
            return self.c2e[cId]
        self.c2e[cId] = str(self.eId)
        self.e2c[str(self.eId)] = cId
        self.eId += 1  
        return self.c2e[cId]
    def convert_cpd2cpd_edge( self, edge ):
        """Currently converts cpd->cpd edge to metabolite->midmarker->metabolite segments
        with the side effect that a new midpoint escher node is created"""
        return self.convert_knot2knot_edge( edge )
    
    def convert_knot2knot_edge( self, edge ):
        """Returns multi->midmarker segment and mid->multimarker segment with new segment id 
        that does not correspond to any edge id
        Side effect: Creates new midmarker node with node_id of edge_id"""
        self.escher.add_node(
            escher_node( 
                self.convert_id(
                    edge.get_edge_id()),
                node_type='midmarker',
                x=edge.get_mid_x(),
                y=edge.get_mid_y()))
        multi2mid = escher_segment( self.convert_id( edge.get_edge_id() + '_multi2mid' ),
                                    from_node_id=self.convert_id( edge.get_source().get_node_id() ),
                                    to_node_id = self.convert_id( edge.get_edge_id() ),
                                    b1=None,
                                    b2=None )

        
        mid2multi = escher_segment( self.convert_id( edge.get_edge_id() + "_mid2multi" ),
                                    from_node_id=self.convert_id( edge.get_edge_id() ),
                                    to_node_id=self.convert_id( edge.get_target().get_node_id() ),
                                    b1=None,
                                    b2=None)
        segments = {}
        segments.update( multi2mid.to_dict() )
        segments.update( mid2multi.to_dict() )
        return segments
    
    def convert_edge2segment( self, edge ):
        return escher_segment( self.convert_id( edge.get_edge_id() ),
                            from_node_id=self.convert_id( edge.get_source().get_node_id() ),
                            to_node_id = self.convert_id(edge.get_target().get_node_id() ),
                            b1=None,
                            b2=None ).to_dict()                
                
    def convert_edges_to_segments( self, edges ):
        segments = {}
        for edge in edges:
            if edge.get_source().is_metabolite_node() and edge.get_target().is_metabolite_node():
                segments.update( self.convert_cpd2cpd_edge( edge ))
            elif edge.get_source().is_knot_node() and edge.get_target().is_knot_node():
                segments.update( self.convert_knot2knot_edge( edge ))
            else:
                segments.update( self.convert_edge2segment( edge ) )
        return segments
            
    def convert_knot_node( self, cynode ):
        if cynode.is_knot_node():
            return escher_node(
                node_id=self.convert_id( cynode.get_node_id()), 
                node_type='multimarker',
                x=cynode.get_x(),
                y=cynode.get_y()
                )
            
    def convert_metabolite_node( self, cynode ):
        if cynode.is_metabolite_node():
            return escher_node( 
                    self.convert_id( cynode.get_node_id( )),
                    bigg_id = cynode.get_frame_id(),
                    label_x = cynode.get_x() ,
                    label_y= cynode.get_y(),
                    name = cynode.get_name(),
                    node_is_primary= True,
                    node_type='metabolite',
                    x=cynode.get_x(),
                    y=cynode.get_y())
    
    def convert_metabolites_of_reaction( self, cyrxn ):
        metabolites = []
        for cpd in cyrxn.get_left( ):
            if cpd and cpd.is_metabolite_node():
                metabolites.append( dict(bigg_id=cpd.get_frame_id(),
                                     coefficient=-1))
        for cpd in cyrxn.get_right( ):
            if cpd and cpd.is_metabolite_node():
                metabolites.append( dict( bigg_id=cpd.get_frame_id(),
                                    coefficient=1))
        return metabolites
                                            

    def convert_reaction( self, cyrxn ): 
            return escher_reaction( self.convert_id( cyrxn.get_frame_id() ),
                      bigg_id= cyrxn.get_frame_id(),
                      genes =  cyrxn.get_genes(),
                      label_x =  cyrxn.get_mid_x(  ),
                      label_y =  cyrxn.get_mid_y(  ),
                      metabolites =  self.convert_metabolites_of_reaction(  cyrxn ),
                      name =  cyrxn.get_name(  ),
                      reversibility =  cyrxn.get_rxn_reversibility(  ),
                      segments =  self.convert_edges_to_segments(cyrxn.get_edges() ) )

    def convert(self ): 
        
        
        for cynode in self.cy.get_metabolites():
                self.escher.add_node( self.convert_metabolite_node( cynode ))
        for cynode in self.cy.get_knots():
                self.escher.add_node( self.convert_knot_node( cynode ))
        return self.escher.to_dict()

In [75]:
c2e = cytoscape2escher(abc_cytoscape)

In [42]:
print(cmodel)

In [47]:
c2e

In [48]:
b = escher.Builder(map_json=json.dumps(c2e),
                  identifiers_on_map='name',
                  show_gene_reaction_rules=True)
b.display_in_notebook()

TypeError: Object of type cytoscape2escher is not JSON serializable

In [84]:
c2e = cytoscape2escher(abc_cytoscape)
abc_c2e = c2e.convert()


with open('ABC_c2e.json','w') as out:
    json.dump( abc_c2e, out, indent=4, sort_keys=True )
with open('ABC_map_pp.json','w') as out:
    json.dump( abc_escher, out, indent=4, sort_keys=True)
# jsonschema.validate(abc_c2e,escher_schema)
# import escher.validate 
# escher.validate.validate_map(abc_c2e)